In [7]:
from pathlib import Path

import polars as pl

In [10]:
ll = []
for fn in Path("../data/occurences/").glob("*.parquet"):
    df = pl.read_parquet(fn)
    ll.append(df)
    print(fn, len(df), df["name"].n_unique())

../data/occurences/Warbreaker - Brandon Sanderson.parquet 13998 133
../data/occurences/Elantris - Brandon Sanderson.parquet 23472 208
../data/occurences/Mistborn_ The Final Empire - Brandon Sanderson.parquet 7981 76


In [ ]:
13998 + 23472 + 7981

45451

In [11]:
fulldf = pl.concat(ll)
print(len(fulldf), fulldf["name"].n_unique())

45451 208


In [ ]:
(fulldf.group_by("name").agg(pl.col("series").n_unique().alias("len")).filter(pl.col("len") > 1))

name,len
str,u32


In [ ]:
(
    fulldf.with_columns(
        _id=pl.concat_str("series", "name", separator="_"),
    )
    .group_by("_id", "series", "name")
    .agg(pl.len().alias("occurrences"))
    .sort("_id", "series", "name")
)

_id,series,name,chapter_id,occurrences
str,str,str,i64,u32
"""elantris_Adien""","""elantris""","""Adien""",5,6
"""elantris_Adien""","""elantris""","""Adien""",8,2
"""elantris_Adien""","""elantris""","""Adien""",29,2
"""elantris_Adien""","""elantris""","""Adien""",58,1
"""elantris_Adien""","""elantris""","""Adien""",60,9
…,…,…,…,…
"""warbreaker_Yarda""","""warbreaker""","""Yarda""",10,2
"""warbreaker_Yarda""","""warbreaker""","""Yarda""",18,2
"""warbreaker_Yesteel""","""warbreaker""","""Yesteel""",52,2
